In [16]:
# Load pandas
import pandas as pd
import shutil, os
import copy

In [17]:
FILE_PATH = '../Ear_images/Ear_images/'

# Read CSV file into DataFrame df
df = pd.read_csv('../Patient_Diagnosis.csv', encoding='utf-8')

filelist = pd.DataFrame(data=[], columns=['Image ID', 'fileid', 'Ear', 'filename'])

for files_name in os.listdir(FILE_PATH):
    result_data = dict()
    (LOG_NAME, ext) = os.path.splitext(files_name)
    names = LOG_NAME.split("_")
    result_data['Image ID'] = names[0]
    result_data['fileid'] = names[1]
    result_data['filename'] = files_name
    result_data['Ear'] = names[2].split(" ")[0]
    filelist = filelist.append(result_data, ignore_index=True)

In [19]:
display(filelist.head())

,Image ID,fileid,Ear,filename
0,6049cd476a52ac001d238542,64a3794c6d814f2aaeee7fd7,Right,6049cd476a52ac001d238542_64a3794c6d814f2aaeee7...
1,6049cd476a52ac001d238542,68bb9a3a3a224ce9bf4c8602,Left,6049cd476a52ac001d238542_68bb9a3a3a224ce9bf4c8...
2,6049cd50f255de001d4efb90,27202710f6fb4fed961c0ec7,Left,6049cd50f255de001d4efb90_27202710f6fb4fed961c0...
3,6049cd50f255de001d4efb90,30fa378e100d4f4284ec10b8,Right,6049cd50f255de001d4efb90_30fa378e100d4f4284ec1...
4,6049cd56a69a81001d8cd215,66de60eadc1b49a597a6e2ee,Right,6049cd56a69a81001d8cd215_66de60eadc1b49a597a6e...


In [21]:
column_names = ['Patient ID', 'Primary Diagnosis', 'Provisional Diagnosis (Name(Ear Type))', 'Image ID']
train_df = df[column_names]
display(train_df.head())

,Patient ID,Primary Diagnosis,Provisional Diagnosis (Name(Ear Type)),Image ID
0,71191,ASOM,ASOM (Left),611e67198b9927003791d9d8
1,71184,Impacted Wax,Impacted Wax (Left),611e67113b8e32003ab7db00
2,71185,ASOM,ASOM (Left),611e6706f6b8510024b0e4c8
3,71186,ASOM,ASOM (Right),611e66ff29b1e0001e271262
4,71187,ASOM,ASOM (Right),611e66f7d6434c002b7b9cf8


In [22]:
def LeftOrRight(x):
    if x.__contains__('(Both)'):
        return 'Both'
    elif x.__contains__('(Right)'):
        return 'Right'
    elif x.__contains__('(Left)'):
        return 'Left'
    return 'Both'

In [23]:
train_df["Ear"] = train_df[column_names[2]].apply(LeftOrRight)
train_df['Primary Diagnosis Text'] = train_df[column_names[1]]
train_df[column_names[1]] = train_df[column_names[1]].apply(lambda x: 1 if x != 'None Of Above' else 0)
display(train_df.head())

<ipython-input-23-bf26bc7b120d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["Ear"] = train_df[column_names[2]].apply(LeftOrRight)
<ipython-input-23-bf26bc7b120d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Primary Diagnosis Text'] = train_df[column_names[1]]
<ipython-input-23-bf26bc7b120d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

,Patient ID,Primary Diagnosis,Provisional Diagnosis (Name(Ear Type)),Image ID,Ear,Primary Diagnosis Text
0,71191,1,ASOM (Left),611e67198b9927003791d9d8,Left,ASOM
1,71184,1,Impacted Wax (Left),611e67113b8e32003ab7db00,Left,Impacted Wax
2,71185,1,ASOM (Left),611e6706f6b8510024b0e4c8,Left,ASOM
3,71186,1,ASOM (Right),611e66ff29b1e0001e271262,Right,ASOM
4,71187,1,ASOM (Right),611e66f7d6434c002b7b9cf8,Right,ASOM


### Separate the Both from Left Right List

In [24]:
both_df = train_df.loc[train_df['Ear'] == 'Both']
display(both_df.head())

,Patient ID,Primary Diagnosis,Provisional Diagnosis (Name(Ear Type)),Image ID,Ear,Primary Diagnosis Text
5,71188,1,ASOM (Both),611e66ebdb5c8b003663c4ee,Both,ASOM
8,2750361,0,None Of Above,611e3e9ed6434c002b7b9cda,Both,None Of Above
9,2750362,0,None Of Above,611e3e97cbf720001d91394a,Both,None Of Above
10,2750363,0,None Of Above,611e3e92e7e3eb002ce4ab5d,Both,None Of Above
11,2750358,0,None Of Above,611e3e8dd6434c002b7b9ccb,Both,None Of Above


### Remove the both from  Left Right List

In [25]:
train_df = train_df.drop(train_df[train_df['Ear'] == 'Both'].index)
for i, both_row in both_df.iterrows():
    right_row = copy.deepcopy(both_row)
    right_row['Ear'] = 'Right'
    train_df = train_df.append(right_row, ignore_index=True)
    
    left_row = copy.deepcopy(both_row)
    left_row['Ear'] = 'Left'
    train_df = train_df.append(left_row, ignore_index=True)

In [26]:
new_train_df = train_df.merge(filelist, on=['Image ID', 'Ear'], how='inner')
# new_train_df = train_df
display(new_train_df.head())

,Patient ID,Primary Diagnosis,Provisional Diagnosis (Name(Ear Type)),Image ID,Ear,Primary Diagnosis Text,fileid,filename
0,71191,1,ASOM (Left),611e67198b9927003791d9d8,Left,ASOM,a322a55421dc4a5e90390a8e,611e67198b9927003791d9d8_a322a55421dc4a5e90390...
1,71184,1,Impacted Wax (Left),611e67113b8e32003ab7db00,Left,Impacted Wax,47160fa7d9844e0ab4473b58,611e67113b8e32003ab7db00_47160fa7d9844e0ab4473...
2,71185,1,ASOM (Left),611e6706f6b8510024b0e4c8,Left,ASOM,5844ba3df51244d7a5e8acd0,611e6706f6b8510024b0e4c8_5844ba3df51244d7a5e8a...
3,71186,1,ASOM (Right),611e66ff29b1e0001e271262,Right,ASOM,9b21bf62ed9c44819dac459e,611e66ff29b1e0001e271262_9b21bf62ed9c44819dac4...
4,71187,1,ASOM (Right),611e66f7d6434c002b7b9cf8,Right,ASOM,47d933e0d3eb420992840483,611e66f7d6434c002b7b9cf8_47d933e0d3eb420992840...


In [27]:
new_train_df.loc[new_train_df['Primary Diagnosis'] == 1].count()

Patient ID                                6787
Primary Diagnosis                         6787
Provisional Diagnosis (Name(Ear Type))    6787
Image ID                                  6787
Ear                                       6787
Primary Diagnosis Text                    6787
fileid                                    6787
filename                                  6787
dtype: int64

In [28]:
new_train_df.loc[new_train_df['Primary Diagnosis'] == 0].count()

Patient ID                                11267
Primary Diagnosis                         11267
Provisional Diagnosis (Name(Ear Type))    11267
Image ID                                  11267
Ear                                       11267
Primary Diagnosis Text                    11267
fileid                                    11267
filename                                  11267
dtype: int64

In [29]:
new_train_df.to_csv('../Patient_Diagnosis_basic.csv', index=False)